In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:/Shubham/Studies/Data Science/NLP/udemy/UPDATED_NLP_COURSE/TextFiles/moviereviews.tsv', sep='\t')

In [3]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [4]:
df.shape

(2000, 2)

In [6]:
# print(df['review'][0])/''

In [7]:
df.isnull().sum()

label      0
review    35
dtype: int64

In [8]:
# Drop NaN values
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [10]:
mystring = 'Hello'
empty = ' '

In [11]:
empty.isspace()

True

In [12]:
# Remove empty string
blanks = []

# index,label, review text
for i,lb,rv in df.itertuples():
    if rv.isspace():
        blanks.append(i)
        

In [14]:
blanks

[57,
 71,
 147,
 151,
 283,
 307,
 313,
 323,
 343,
 351,
 427,
 501,
 633,
 675,
 815,
 851,
 977,
 1079,
 1299,
 1455,
 1493,
 1525,
 1531,
 1763,
 1851,
 1905,
 1993]

In [16]:
df.drop(blanks, inplace=True)

In [17]:
df.shape

(1938, 2)

In [20]:
from sklearn.model_selection import train_test_split

In [18]:
X = df['review']
y = df['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [23]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [24]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [25]:
predictions = text_clf.predict(X_test)

In [26]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [27]:
print(confusion_matrix(y_test, predictions))

[[235  47]
 [ 41 259]]


In [28]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         neg       0.85      0.83      0.84       282
         pos       0.85      0.86      0.85       300

   micro avg       0.85      0.85      0.85       582
   macro avg       0.85      0.85      0.85       582
weighted avg       0.85      0.85      0.85       582



In [29]:
print(accuracy_score(y_test, predictions))

0.8487972508591065


### Spacy

In [2]:
import spacy

In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
# Document is the avg of the vectors of the single words in here
nlp(u"The quick brown fox jumped").vector.shape

(300,)

In [5]:
nlp(u"fox").vector.shape

(300,)

In [6]:
tokens = nlp(u"like love hate")

In [7]:
for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

like like 1.0
like love 0.65790397
like hate 0.6574652
love like 0.65790397
love love 1.0
love hate 0.6393099
hate like 0.6574652
hate love 0.6393099
hate hate 1.0


In [8]:
nlp.vocab.vectors.shape

(20000, 300)

In [9]:
tokens = nlp(u"dog cat nargle")

In [10]:
for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
nargle False 0.0 True


In [11]:
from scipy import spatial

cosine_similarity = lambda vec1,vec2: 1 - spatial.distance.cosine(vec1, vec2)

In [12]:
king = nlp.vocab['king'].vector
man = nlp.vocab['man'].vector
woman = nlp.vocab['woman'].vector

In [13]:
# king - man + woman ----> NEW_VECTOR similar to queen

new_vector = king-man+woman

In [14]:
computed_similarties = []

for word in nlp.vocab:
    if word.has_vector:
        if word.is_lower:
            if word.is_alpha:
                similarity = cosine_similarity(new_vector, word.vector)
                computed_similarties.append((word, similarity))

In [15]:
# Descending ording sorting
computed_similarities = sorted(computed_similarties, key=lambda item:-item[1])

In [16]:
print([t[0].text for t in computed_similarties[:10]])

['croup', 'gorey', 'mcleish', 'datsyuk', 'eradicate', 'meeni', 'traverser', 'mersey', 'romen', 'guthrie']
